# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}
\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}
\begin{equation}
A_{xy} = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/'

fileName = "ainvert.py"
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileNames = ["parrot.bmp", "clock.bmp", "chessboard.bmp", "bart.png", "firetruck.jpg"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

In [ ]:
import ainvert

def bicubic(I, wsp_h, wsp_w):
  h, w = I.shape
  h2, w2 = int(wsp_h * h), int(wsp_w * w)

  def img(I, param):
    i,j = param
    return I[i][j]

  def dx(I, param):
    i,j = param
    h, w = I.shape
    if h-1 == i:
      i = i - 1
    if i == 0:
      i = i + 1
    return (I[i+1][j] - I[i-1][j])/2
  
  def dy(I, param):
    i,j = param
    h, w = I.shape
    if w-1 == j:
      j = j - 1
    if j == 0:
      j = j + 1
    return (I[i][j+1] - I[i][j-1])/2

  def dxy(I, param):
    i,j = param
    h, w = I.shape
    if h-1 == i:
      i = i - 1
    if i == 0:
      i = i + 1
    if w-1 == j:
      j = j - 1
    if j == 0:
      j = j + 1
    return (I[i+1][j+1] - I[i-1][j] - I[i][j-1] + I[i][j])/4
  
  def p(x,y,a):
    left = np.array([1, y, y**2, y**3])
    right = np.array([1, x, x**2, x**3]).transpose()
    return left @ a @ right

  def find_nearest(ii, jj, I, wsp_h, wsp_w):
    h3, w3 = I.shape
    i = int(ii/wsp_h)
    j = int(jj/wsp_w)

    A = (i, j)
    B = ((i+1) if i != h-1 else i, j)
    C = ((i+1) if i != h-1 else i, (j+1) if j != w-1 else j)
    D = (i, (j+1) if j != w-1 else j)

    x = np.array([img(I,A), img(I,B), img(I,D), img(I,C), dx(I, A),dx(I, B),dx(I, D),dx(I, C),\
         dy(I, A),dy(I, B),dy(I, D),dy(I, C),dxy(I, A),dxy(I, B),dxy(I, D),dxy(I, C),])
    
    a = np.array(ainvert.A_invert @ x.transpose())
    return p(ii/wsp_h - i, jj/wsp_w - j, a.reshape((4,4)))

  I = I.astype(np.int64)
  I2 = np.array([ [ 0 for _ in range(w2) ] for _ in range(h2) ])
  for i in range(h2):
    for j in range(w2):
      I2[i][j] = find_nearest(i,j, I, wsp_h, wsp_w)

  return I2


Dane: Obraz o szerokości w2 i wysokości h2

Interpolacja dwusześcienna:<br>
Operacje arytmetyczne:
*   \+ w2\*h2\*19
*   \- w2\*h2\*38
*   \/ w2\*h2\*16 
*   \* w2\*h2\*6
*   Dla operacji \@: 
*     *   \* w2\*h2\*276
*     *   \+ w2\*h2\*255

Operacje dostępu do pamięci:
*   w\*h
*   w2\*h2\*37

Interpolacja dwuliniowa: <br>
Operacje arytmetyczne:
*   \+ w2\*h2\*7
*   \- w2\*h2\*14
*   \* w2\*h2\*8
*   \/ w2\*h2\*4

Operacje dostępu do pamięci:
*   w2\*h2\*5

Wniosek:<br>
Operacji dla interpolacji dwusześciennej jest znacznie więcej. Dla obrazka wejściowego 255x255 mamy złożoność mnożeń sześcienną w interpolacji dwusześciennej, a w interpolacji dwuliniowej nadal kwadrotową.

In [ ]:
I = cv2.imread('chessboard.bmp')         
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY) 
I2 = bicubic(I, 2.5,2.5)
plt.figure(figsize=(I2.shape[0]/100,I2.shape[1]/100), dpi=200)
plt.imshow(I2, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
I = cv2.imread('parrot.bmp')         
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY) 
I2 = bicubic(I, 2.5,2.5)
plt.figure(figsize=(I2.shape[0]/100,I2.shape[1]/100), dpi=200)
plt.imshow(I2, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
I = cv2.imread('parrot.bmp')         
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY) 
I2 = bicubic(I, 0.5, 0.5)
plt.figure(figsize=(I2.shape[0]/100,I2.shape[1]/100), dpi=200)
plt.imshow(I2, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
I = cv2.imread('clock.bmp')         
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY) 
I2 = bicubic(I, 2.5,2.5)
plt.figure(figsize=(I2.shape[0]/100,I2.shape[1]/100), dpi=200)
plt.imshow(I2, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
I = cv2.imread('firetruck.jpg')         
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY) 
I2 = bicubic(I, 2.5,2.5)
plt.figure(figsize=(I2.shape[0]/100,I2.shape[1]/100), dpi=200)
plt.imshow(I2, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()